# Грузим датасет

In [1]:
import pandas as pd
dtypes={
        'session_id': 'Int64',      
        'user_id': 'Int64',
        'is_new_user': 'boolean',
        'country': 'category',
        'platform': 'category',
        'game_id': 'category',
        'session_status': 'category'
    }
data = pd.read_csv('Тестовый датасет.csv', dtype=dtypes, parse_dates=['session_start', 'session_end'], na_values=['', 'None', 'NaN'])
data.head()

,session_id,user_id,is_new_user,country,session_start,session_end,platform,game_id,session_status
0,1,328,True,RU,2025-01-01 01:06:17,2025-01-01 01:27:17,PC,102,completed
1,2,17,False,RU,2025-01-01 00:21:14,2025-01-01 02:36:14,PC,202,completed
2,3,302,True,US,2025-01-01 01:28:51,2025-01-01 02:27:51,Android,103,completed
3,4,391,True,NaN,2025-01-01 00:53:24,2025-01-01 01:34:24,IOS,105,crashed
4,5,374,True,US,2025-01-01 02:02:24,2025-01-01 02:09:24,IOS,201,completed


## Обработка пропусков

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   session_id      2000 non-null   Int64         
 1   user_id         2000 non-null   Int64         
 2   is_new_user     1703 non-null   boolean       
 3   country         1591 non-null   category      
 4   session_start   2000 non-null   datetime64[ns]
 5   session_end     2000 non-null   datetime64[ns]
 6   platform        2000 non-null   category      
 7   game_id         2000 non-null   category      
 8   session_status  2000 non-null   category      
dtypes: Int64(2), boolean(1), category(4), datetime64[ns](2)
memory usage: 79.0 KB


Null буду считать как неизвестно, поэтому чистить их не будем.

In [3]:
na_is_new_user = data['is_new_user'].isna()
unknown_country = data['country'].isna()

## Смотрим, что в категориальных колонках

In [4]:
data['country'].value_counts()

country
RU    480
US    420
DE    282
TR    209
FR    200
Name: count, dtype: int64

In [5]:
data['platform'].value_counts()

platform
IOS        688
PC         657
Android    655
Name: count, dtype: int64

In [6]:
data['session_status'].value_counts()

session_status
completed      1535
crashed         283
interrupted     182
Name: count, dtype: int64

# Расчёт метрик

## Число уникальных пользователей в день

1 подход: считаем пользователя только по началу сессии (не учитываем сессии пересёкшие полночь как две)

In [7]:
data['session_start_date'] = data['session_start'].dt.date
dau_1 = data.groupby('session_start_date')['user_id'].nunique().rename('dau').reset_index()
dau_1

,session_start_date,dau
0,2025-01-01,417
1,2025-01-02,90
2,2025-01-03,1


In [65]:
with pd.ExcelWriter('Ответы на 3 задание.xlsx', engine='openpyxl') as writer:
    dau_1.to_excel(writer, sheet_name='DAU', index=False)

2 подход: учитываем заезд сессии на несколько дней

In [8]:
data['session_length'] = data['session_end'] - data['session_start']
data['session_length'].max()

Timedelta('0 days 06:07:00')

Нет сессии больше дня, значит макс может быть пересечение в полночь

In [9]:
data['session_end_date'] = data['session_end'].dt.date

In [10]:
def expand_session_days(df):
    
    # Находим сессии, пересекающие полночь
    mask_cross_midnight = df['session_start_date'] != df['session_end_date']
    
    # Для простых сессий сразу дата начала
    result = df.loc[~mask_cross_midnight, ['user_id']].copy()
    result['date'] = df.loc[~mask_cross_midnight, 'session_start_date']
    
    # Для междневных сессий добавляем оба дня
    for _, row in df.loc[mask_cross_midnight].iterrows():

        start_date = pd.DataFrame({
            'user_id': [row['user_id']],
            'date': [row['session_start_date']]
        })

        both_dates = pd.concat([
            pd.DataFrame({'user_id': [row['user_id']], 'date': [row['session_start_date']]}), 
            pd.DataFrame({'user_id': [row['user_id']], 'date': [row['session_end_date']]})
            ], ignore_index=True)

        result = pd.concat([result, both_dates], ignore_index=True)
    
    return result.drop_duplicates()

expanded = expand_session_days(data)
daily_dau_full = expanded.groupby('date')['user_id'].nunique().rename('dau').reset_index()
daily_dau_full

,date,dau
0,2025-01-01,417
1,2025-01-02,95
2,2025-01-03,1


In [69]:
with pd.ExcelWriter('Ответы на 3 задание.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    daily_dau_full.to_excel(writer, sheet_name='DAU_более_точный', index=False)

## Количество сессий для каждой платформы

In [11]:
data.session_id.nunique()

2000

Все сессии уникальные

In [12]:
sessions_by_platform = data['platform'].value_counts().rename('total_sessions').reset_index()
sessions_by_platform

,platform,total_sessions
0,IOS,688
1,PC,657
2,Android,655


In [85]:
with pd.ExcelWriter('Ответы на 3 задание.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    sessions_by_platform.to_excel(writer, sheet_name='sessions_by_platform', index=False)

## Средняя длительность сессии для каждой платформы

In [13]:
avg_session_lengths_by_platform = data.groupby('platform')['session_length'].mean().rename('avg_session_length').reset_index()
avg_session_lengths_by_platform['avg_session_length_min'] = (avg_session_lengths_by_platform['avg_session_length'].dt.total_seconds() / 60).round(2)
del avg_session_lengths_by_platform['avg_session_length']
avg_session_lengths_by_platform

C:\Users\M A C H R E A T O R\AppData\Local\Temp\ipykernel_15948\1848277756.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_session_lengths_by_platform = data.groupby('platform')['session_length'].mean().rename('avg_session_length').reset_index()


,platform,avg_session_length_min
0,Android,42.87
1,IOS,47.34
2,PC,43.77


In [93]:
with pd.ExcelWriter('Ответы на 3 задание.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    avg_session_lengths_by_platform.to_excel(writer, sheet_name='avg_session_lengths_by_platform', index=False)

## Доля сессий, приходящихся на новых пользователей

In [14]:
percentages = (data.loc[~na_is_new_user, 'is_new_user'].value_counts(normalize=True) * 100).rename('percentage').reset_index()
percentage_new = percentages[percentages['is_new_user']]
percentage_new

,is_new_user,percentage
1,True,29.712272


In [111]:
with pd.ExcelWriter('Ответы на 3 задание.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    percentage_new.to_excel(writer, sheet_name='percentage_new', index=False)

## Медиана длительности сессий отдельно у старых и новых пользователей

In [15]:
session_length_medians = (data.groupby('is_new_user')['session_length'].median().dt.total_seconds() / 60).rename('median_session_length_min').reset_index()
session_length_medians

,is_new_user,median_session_length_min
0,False,32.0
1,True,31.5


In [123]:
with pd.ExcelWriter('Ответы на 3 задание.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    session_length_medians.to_excel(writer, sheet_name='session_length_medians', index=False)

## Сумарное отыгранное время в стране с кодом 'RU'

In [16]:
sum_by_RU = round((data.loc[data['country'] == 'RU', 'session_length'].sum().total_seconds() / 3600), 2)
sum_by_RU_table = pd.DataFrame({
    'Суммарное отыгранное время в стране с кодом \'RU\' (часы)': [sum_by_RU]
})
sum_by_RU_table

,Суммарное отыгранное время в стране с кодом 'RU' (часы)
0,359.82


In [151]:
with pd.ExcelWriter('Ответы на 3 задание.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    sum_by_RU_table.to_excel(writer, sheet_name='total_session_length_by_RU', index=False)

# Интерпретация метрик

## Тесты на значимость (было лень самому писать, так что просто написал промпт для ии)

In [19]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')  # скрываем предупреждения о нормальности для больших выборок

# Подготовка данных: длительность в минутах
data['session_length_min'] = (data['session_end'] - data['session_start']).dt.total_seconds() / 60
data_clean = data.dropna(subset=['session_length_min', 'platform'])

# ============================================================================
# 1. Тест хи-квадрат: равномерно ли распределены сессии по платформам?
# ============================================================================
print("="*70)
print("1. Тест хи-квадрат для распределения сессий по платформам")
print("="*70)

observed = data_clean['platform'].value_counts().sort_index()
expected = np.full(len(observed), observed.sum() / len(observed))  # равномерное распределение

chi2_stat, p_value_chi2 = stats.chisquare(f_obs=observed, f_exp=expected)

print(f"\nНаблюдаемые частоты:\n{observed}")
print(f"\nОжидаемые частоты (равномерное распределение): {expected[0]:.1f} на платформу")
print(f"\nχ² = {chi2_stat:.2f}, p-value = {p_value_chi2:.4f}")
print(f"Вывод: {'Распределение НЕ равномерное (статистически значимо)' if p_value_chi2 < 0.05 else 'Распределение равномерное'}")

# ============================================================================
# 2. Проверка предпосылок для параметрических тестов (для справки)
# ============================================================================
print("\n" + "="*70)
print("2. Проверка нормальности и гомогенности дисперсий")
print("="*70)

platforms = ['Android', 'IOS', 'PC']
samples = {plat: data_clean[data_clean['platform'] == plat]['session_length_min'] for plat in platforms}

# Нормальность (ограничиваем выборку до 5000 для корректности теста Шапиро)
print("\nТест Шапиро-Уилка (нормальность, выборка ≤5000):")
for plat in platforms:
    sample = samples[plat].sample(n=min(5000, len(samples[plat])), random_state=42)
    _, p = stats.shapiro(sample)
    print(f"  {plat:6s}: p = {p:.4f} → {'НЕ нормальное' if p < 0.05 else 'нормальное'}")

# Гомогенность дисперсий
_, p_levene = stats.levene(*[samples[plat] for plat in platforms])
print(f"\nТест Левена (гомогенность дисперсий): p = {p_levene:.4f} → {'дисперсии НЕ равны' if p_levene < 0.05 else 'дисперсии равны'}")

# ============================================================================
# 3. Критерий Краскела-Уоллиса (непараметрический аналог ANOVA)
# ============================================================================
print("\n" + "="*70)
print("3. Критерий Краскела-Уоллиса для длительности сессий")
print("="*70)

kw_stat, p_value_kw = stats.kruskal(
    samples['Android'],
    samples['IOS'],
    samples['PC']
)

print(f"\nH = {kw_stat:.2f}, p-value = {p_value_kw:.4f}")
print(f"Вывод: {'Есть статистически значимые различия между платформами' if p_value_kw < 0.05 else 'Различий между платформами нет'}")

# ============================================================================
# 4. Попарные сравнения с поправкой Бонферрони (если общий тест значим)
# ============================================================================
if p_value_kw < 0.05:
    print("\n" + "="*70)
    print("4. Попарные сравнения (Mann-Whitney U с поправкой Бонферрони)")
    print("="*70)
    
    pairs = [('Android', 'IOS'), ('Android', 'PC'), ('IOS', 'PC')]
    alpha = 0.05 / len(pairs)  # поправка Бонферрони
    
    results = []
    for plat1, plat2 in pairs:
        u_stat, p_raw = stats.mannwhitneyu(samples[plat1], samples[plat2], alternative='two-sided')
        p_adj = p_raw * len(pairs)  # ручная поправка
        mean1 = samples[plat1].mean()
        mean2 = samples[plat2].mean()
        diff = mean1 - mean2
        
        significant = p_adj < alpha
        results.append({
            'Сравнение': f"{plat1} vs {plat2}",
            'U': u_stat,
            'p-value (сырой)': p_raw,
            'p-value (скорр.)': p_adj,
            'Δ среднее (мин)': diff,
            'Значимо': 'Да' if significant else 'Нет'
        })
    
    results_df = pd.DataFrame(results)
    print("\n", results_df.to_string(index=False))
    
    print(f"\nПорог значимости после поправки Бонферрони: α = {alpha:.4f}")
    print("\nИнтерпретация:")
    for _, row in results_df.iterrows():
        if row['Значимо'] == 'Да':
            plat1, plat2 = row['Сравнение'].split(' vs ')
            diff = row['Δ среднее (мин)']
            direction = f"{plat1} дольше на {diff:.1f} мин" if diff > 0 else f"{plat2} дольше на {abs(diff):.1f} мин"
            print(f"  • {row['Сравнение']}: {direction} (p={row['p-value (скорр.)']:.4f})")
        else:
            print(f"  • {row['Сравнение']}: различия НЕ значимы")

# ============================================================================
# 5. Дополнительно: доверительные интервалы для средних (бутстреп)
# ============================================================================
print("\n" + "="*70)
print("5. Доверительные интервалы для средней длительности (95% ДИ, бутстреп)")
print("="*70)

def bootstrap_ci(data, n_boot=1000, alpha=0.05):
    means = [np.mean(np.random.choice(data, size=len(data), replace=True)) 
             for _ in range(n_boot)]
    return np.percentile(means, [100*alpha/2, 100*(1-alpha/2)])

for plat in platforms:
    sample = samples[plat].values
    ci_low, ci_high = bootstrap_ci(sample, n_boot=1000)
    mean_val = sample.mean()
    print(f"{plat:6s}: {mean_val:.2f} мин (95% ДИ: [{ci_low:.2f}, {ci_high:.2f}])")

1. Тест хи-квадрат для распределения сессий по платформам

Наблюдаемые частоты:
platform
Android    655
IOS        688
PC         657
Name: count, dtype: int64

Ожидаемые частоты (равномерное распределение): 666.7 на платформу

χ² = 1.03, p-value = 0.5984
Вывод: Распределение равномерное

2. Проверка нормальности и гомогенности дисперсий

Тест Шапиро-Уилка (нормальность, выборка ≤5000):
  Android: p = 0.0000 → НЕ нормальное
  IOS   : p = 0.0000 → НЕ нормальное
  PC    : p = 0.0000 → НЕ нормальное

Тест Левена (гомогенность дисперсий): p = 0.0218 → дисперсии НЕ равны

3. Критерий Краскела-Уоллиса для длительности сессий

H = 0.66, p-value = 0.7186
Вывод: Различий между платформами нет

5. Доверительные интервалы для средней длительности (95% ДИ, бутстреп)
Android: 42.87 мин (95% ДИ: [39.60, 46.01])
IOS   : 47.34 мин (95% ДИ: [43.60, 50.96])
PC    : 43.77 мин (95% ДИ: [40.80, 47.24])


## DAU

Видно, что за 3 дня DAU стремительно падает. Такое резкое снижение может указывать на:
- Завершение временной акции или события (например, новогодней кампании)
- Проблемы с доступностью сервиса после 1 января
- Естественный спад активности после праздников

**Рекомендация:** провести анализ причин падения активности и разработать стратегию удержания пользователей

## Распределение сессий по платформам

Пользователи равномерно распределены между всеми платформами, хотя у IOS-платформы есть небольшое преимущество (статичтически незначимое). Это свидетельствует об успешной адаптации продукта под все основные платформы

## Средняя длительность сессий по платформам

Средняя длительность тоже равномероно распределена. Но чисто по цифрам видно, что на IOS немного больше, так что мб имеет смысл посмотреть на большем датасете различия.

**Рекомендация:** проверить разницу на большом датасете

## Доля новых пользователей

Высокая доля новых пользователей (почти 30%) свидетельствует об:
- Эффективных маркетинговых кампаниях
- Привлекательном продукте с хорошей вирусностью
- Успешных реферальных программах
  
Это положительный показатель, но важно отслеживать конверсию новых пользователей в активных

## Медиана длительности сессий по категориям пользователей

Практически одинаковая вовлеченность новых и старых пользователей. Это говорит о:
- Успешном онбординге новых пользователей
- Хорошем первом впечатлении от сервиса
- Эффективной адаптации новых пользователей к игровому процессу
  
**Рекомендация:** продолжать поддерживать текущий уровень онбординга и искать способы дальнейшего повышения вовлеченности новых пользователей

## Суммарное время игры в России

Очевидно для сравнения имеет смысл посчитать для остальных регионов.

In [31]:
sum_by_country = (data.groupby('country')['session_length'].sum()).rename('total_length').reset_index().sort_values(by='total_length', ascending=False)
sum_by_country

,country,total_length
2,RU,14 days 23:49:00
4,US,12 days 20:45:00
0,DE,8 days 07:39:00
3,TR,6 days 06:00:00
1,FR,5 days 23:01:00


Россия является одним из ключевых рынков для сервиса. Но тут нужно учитывать использование впн и тд.

# Дополнительные метрики

**1. Удержание новых пользователей (D1/D7 retention)**  
Показывает, насколько продукт «цепляет» после первого контакта. Критично для геймдева: если новые пользователи не возвращаются на 2-й день — проблема в онбординге или первом впечатлении.

**2. % завершённых сессий (`completed` / всего)**  
Отражает стабильность сервиса и качество пользовательского опыта. Высокий % крашей/обрывов = технические проблемы или сложный интерфейс.

**3. Средняя длительность сессии по платформам**  
Помогает выявить платформу с лучшей вовлечённостью (в данных — iOS) и скопировать успешные практики на другие платформы.

**4. Активность в часы пик (распределение сессий по времени)**  
Позволяет оптимизировать нагрузку на серверы и планировать маркетинговые активности в моменты пиковой аудитории.